In [21]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [22]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [23]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

41951


In [24]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [25]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [26]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

In [27]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE name='cid'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,65,cid,None,None,None,cenH3_(Plants)


# To Do

## <span style="color:green">Set `Taxonomic span` of cenH3 to Eukaryotes, 2759</span>

## <span style="color:green">Set `Alternate names` CENP-A, Cse4, HCP-3, CNP1, HTR12, CNA1, cid to their nodes</span>

1. **CENP-A** (Centromere Protein A) — стандартное название у млекопитающих, включая человека.
2. **Cse4** and **CNP1** — названия в дрожжах *Saccharomyces cerevisiae*.  
3. **HCP-3** — вариант у нематоды *Caenorhabditis elegans*.  
4. **CNA1** — альтернативное название у *Tetrahymena thermophila* (SAR).  
5. **HTR12** — обозначение у растения *Arabidopsis thaliana*.  
6. **cid** — название у дрозофилы (*Drosophila melanogaster*).  

**cenH3:** CENP-A, Cse4, HCP-3, CNP1, HTR12, CNA1, cid

**cenH3_(Animals):** CENP-A, HCP-3, cid

**cenH3_(Mammalia):** CENP-A

**cenH3_(Homo_sapiens):** CENP-A

**cenH3_(Fungi):** Cse4, CNP1

**cenH3_(Plants):** HTR12


# Set `Taxonomic span` of cenH3 to Eukaryotes, 2759

In [7]:
query = (
    "SELECT * FROM histone "
    "WHERE id='cenH3'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,cenH3,variant_group,null,null,96,H3


In [9]:
query = f"UPDATE histone SET taxonomic_span='Eukaryotes', taxonomic_span_id=2759 WHERE id='cenH3'"
print(query)
cursor.execute(query)

UPDATE histone SET taxonomic_span='Eukaryotes', taxonomic_span_id=2759 WHERE id='cenH3'


In [10]:
query = (
    "SELECT * FROM histone "
    "WHERE id='cenH3'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,cenH3,variant_group,Eukaryotes,2759,96,H3


In [11]:
# Make sure data is committed to the database
conn.commit()

# Set `Alternate names` CENP-A, Cse4, HCP-3, CNP1, HTR12, CNA1, cid to their nodes

1. **CENP-A** (Centromere Protein A) — стандартное название у млекопитающих, включая человека.
2. **Cse4** and **CNP1** — названия в дрожжах *Saccharomyces cerevisiae*.  
3. **HCP-3** — вариант у нематоды *Caenorhabditis elegans*.  
4. **CNA1** — альтернативное название у *Tetrahymena thermophila* (SAR).  
5. **HTR12** — обозначение у растения *Arabidopsis thaliana*.  
6. **cid** — название у дрозофилы (*Drosophila melanogaster*).  

**cenH3:** CENP-A, Cse4, HCP-3, CNP1, HTR12, CNA1, cid

**cenH3_(Animals):** CENP-A, HCP-3, cid

**cenH3_(Mammalia):** CENP-A

**cenH3_(Homo_sapiens):** CENP-A

**cenH3_(Fungi):** Cse4, CNP1

**cenH3_(Plants):** HTR12

In [30]:
query = (
    "SELECT * FROM histone h LEFT JOIN alternative_name an "
    "ON h.id = an.histone "
    "WHERE h.id='cenH3'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,name,taxonomy,gene,splice,histone
0,cenH3,variant_group,Eukaryotes,2759,96,H3,None,None,None,None,None,None


In [31]:
query = (
    "SELECT * FROM histone h LEFT JOIN alternative_name an "
    "ON h.id = an.histone "
    "WHERE h.id='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,name,taxonomy,gene,splice,histone
0,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,59,CENP-A,None,None,None,cenH3_(Plants)
1,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,60,Cse4,None,None,None,cenH3_(Plants)
2,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,61,HCP-3,None,None,None,cenH3_(Plants)
3,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,62,CNP1,None,None,None,cenH3_(Plants)
4,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,63,HTR12,None,None,None,cenH3_(Plants)
5,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,64,CNA1,None,None,None,cenH3_(Plants)
6,cenH3_(Plants),variant,Eukaryotes,2759,233,cenH3,65,cid,None,None,None,cenH3_(Plants)


In [32]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,59,CENP-A,None,None,None,cenH3_(Plants)
1,60,Cse4,None,None,None,cenH3_(Plants)
2,61,HCP-3,None,None,None,cenH3_(Plants)
3,62,CNP1,None,None,None,cenH3_(Plants)
4,63,HTR12,None,None,None,cenH3_(Plants)
5,64,CNA1,None,None,None,cenH3_(Plants)
6,65,cid,None,None,None,cenH3_(Plants)


In [33]:
for i in range(59, 66):
    query = f"UPDATE alternative_name SET histone='cenH3' WHERE id='{i}'"
    print(query)
    cursor.execute(query)

UPDATE alternative_name SET histone='cenH3' WHERE id='59'
UPDATE alternative_name SET histone='cenH3' WHERE id='60'
UPDATE alternative_name SET histone='cenH3' WHERE id='61'
UPDATE alternative_name SET histone='cenH3' WHERE id='62'
UPDATE alternative_name SET histone='cenH3' WHERE id='63'
UPDATE alternative_name SET histone='cenH3' WHERE id='64'
UPDATE alternative_name SET histone='cenH3' WHERE id='65'


In [34]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,59,CENP-A,None,None,None,cenH3
1,60,Cse4,None,None,None,cenH3
2,61,HCP-3,None,None,None,cenH3
3,62,CNP1,None,None,None,cenH3
4,63,HTR12,None,None,None,cenH3
5,64,CNA1,None,None,None,cenH3
6,65,cid,None,None,None,cenH3


## Add cenH3_(Animals): CENP-A, HCP-3, cid

In [36]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Animals)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone


In [38]:
alternate_names = ["CENP-A", "HCP-3", "cid"]
for an in alternate_names:
    data_alternate_name = {
        "name": an,
        "taxonomy": None,
        "gene": None,
        "splice": None,
        "histone": "cenH3_(Animals)",
    }
    cursor.execute(add_alternate_names, data_alternate_name)

In [39]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Animals)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,91,CENP-A,None,None,None,cenH3_(Animals)
1,92,HCP-3,None,None,None,cenH3_(Animals)
2,93,cid,None,None,None,cenH3_(Animals)


## Add cenH3_(Mammalia): CENP-A

In [40]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone


In [41]:
alternate_names = ["CENP-A"]
for an in alternate_names:
    data_alternate_name = {
        "name": an,
        "taxonomy": None,
        "gene": None,
        "splice": None,
        "histone": "cenH3_(Mammalia)",
    }
    cursor.execute(add_alternate_names, data_alternate_name)

In [42]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,94,CENP-A,None,None,None,cenH3_(Mammalia)


## Add cenH3_(Homo_sapiens): CENP-A

In [43]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone


In [44]:
alternate_names = ["CENP-A"]
for an in alternate_names:
    data_alternate_name = {
        "name": an,
        "taxonomy": None,
        "gene": None,
        "splice": None,
        "histone": "cenH3_(Homo_sapiens)",
    }
    cursor.execute(add_alternate_names, data_alternate_name)

In [45]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,95,CENP-A,None,None,None,cenH3_(Homo_sapiens)


## Add cenH3_(Fungi): Cse4, CNP1

In [46]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Fungi)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone


In [47]:
alternate_names = ["Cse4", "CNP1"]
for an in alternate_names:
    data_alternate_name = {
        "name": an,
        "taxonomy": None,
        "gene": None,
        "splice": None,
        "histone": "cenH3_(Fungi)",
    }
    cursor.execute(add_alternate_names, data_alternate_name)

In [48]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Fungi)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,96,Cse4,None,None,None,cenH3_(Fungi)
1,97,CNP1,None,None,None,cenH3_(Fungi)


## Add cenH3_(Plants): HTR12

In [49]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone


In [50]:
alternate_names = ["HTR12"]
for an in alternate_names:
    data_alternate_name = {
        "name": an,
        "taxonomy": None,
        "gene": None,
        "splice": None,
        "histone": "cenH3_(Plants)",
    }
    cursor.execute(add_alternate_names, data_alternate_name)

In [51]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cenH3_(Plants)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,98,HTR12,None,None,None,cenH3_(Plants)


In [52]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [53]:
cursor.close()
conn.close()
tunnel.stop()